# Using FABRIC GPUs

Your compute nodes can include GPUs. These devices are made available as FABRIC components and can be added to your nodes like any other component.

This example notebook will demonstrate how to reserve and use GPU devices on FABRIC.

## Configure the Environment

In [ ]:
import os
from fabrictestbed.slice_manager import SliceManager, Status, SliceState
import json

In [ ]:
ssh_key_file_priv=os.environ['HOME']+"/.ssh/id_rsa"
ssh_key_file_pub=os.environ['HOME']+"/.ssh/id_rsa.pub"

ssh_key_pub = None
with open (ssh_key_file_pub, "r") as myfile:
    ssh_key_pub=myfile.read()
    ssh_key_pub=ssh_key_pub.strip()

In [ ]:
credmgr_host = os.environ['FABRIC_CREDMGR_HOST']
print(f"FABRIC Credential Manager   : {credmgr_host}")

orchestrator_host = os.environ['FABRIC_ORCHESTRATOR_HOST']
print(f"FABRIC Orchestrator         : {orchestrator_host}")

## Create Slice Manager Object

In [ ]:
slice_manager = SliceManager(oc_host=orchestrator_host, 
                             cm_host=credmgr_host ,
                             project_name='all', 
                             scope='all')

# Initialize the slice manager
slice_manager.initialize()

## Create a Node

The cell below creates a slice that contains a single node. The node includes a GPU component.

### Set the Slice Name and FABRIC Site

In [ ]:
from fabrictestbed.slice_editor import ComponentType

slice_name="MySlice"
site_name="RENC"
node_name='Node1'

image = 'default_ubuntu_20'
image_type = 'qcow2'
cores = 2
ram = 16
disk = 100

component_type = ComponentType.GPU
component_model = 'Tesla T4' # Options 'Tesla T4' or (coming soon) 'RTX6000'
component_name = 'gpu1'

In [ ]:
from fabrictestbed.slice_editor import ExperimentTopology, Capacities
# Create topology
t = ExperimentTopology()

# Add node
n1 = t.add_node(name=node_name, site=site_name)

# Set capacities
cap = Capacities()
cap.set_fields(core=cores, ram=ram, disk=disk)

# Set Properties
n1.set_properties(capacities=cap, image_type=image_type, image_ref=image)

# Add the PCI GPU device - Options 'Tesla T4' or (coming soon) 'RTX6000'
n1.add_component(ctype=component_type, model=component_model, name=component_name)

# Generate Slice Graph
slice_graph = t.serialize()

# Request slice from Orchestrator
return_status, slice_reservations = slice_manager.create(slice_name=slice_name, 
                                            slice_graph=slice_graph, 
                                            ssh_key=ssh_key_pub)

if return_status == Status.OK:
    slice_id = slice_reservations[0].get_slice_id()
    print("Submitted slice creation request. Slice ID: {}".format(slice_id))
else:
    print(f"Failure: {slice_reservations}")

## Get the Slice

Retrieve the node information and save the management IP addresses.

In [ ]:
import time
def wait_for_slice(slice,timeout=180,interval=10,progress=False):
    timeout_start = time.time()

    if progress: print("Waiting for slice .", end = '')
    while time.time() < timeout_start + timeout:
        return_status, slices = slice_manager.slices(excludes=[SliceState.Dead,SliceState.Closing])

        if return_status == Status.OK:
            slice = list(filter(lambda x: x.slice_name == slice_name, slices))[0]
            if slice.slice_state == "StableOK":
                if progress: print(" Slice state: {}".format(slice.slice_state))
                return slice
            if slice.slice_state == "Closing" or slice.slice_state == "Dead":
                if progress: print(" Slice state: {}".format(slice.slice_state))
                return slice    
        else:
            print(f"Failure: {slices}")
        
        if progress: print(".", end = '')
        time.sleep(interval)
    
    if time.time() >= timeout_start + timeout:
        if progress: print(" Timeout exceeded ({} sec). Slice: {} ({})".format(timeout,slice.slice_name,slice.slice_state))
        return slice    


return_status, slices = slice_manager.slices(excludes=[SliceState.Dead,SliceState.Closing])

if return_status == Status.OK:
    slice = list(filter(lambda x: x.slice_name == slice_name, slices))[0]
    slice = wait_for_slice(slice, progress=True)

print()
print("Slice Name : {}".format(slice.slice_name))
print("ID         : {}".format(slice.slice_id))
print("State      : {}".format(slice.slice_state))
print("Lease End  : {}".format(slice.lease_end))

## Get the Node

Retrieve the node information and save the management IP address.


In [ ]:
return_status, experiment_topology = slice_manager.get_slice_topology(slice_object=slice)

node = experiment_topology.nodes[node_name]

management_ip = str(node.get_property(pname='management_ip'))
print("Node Name        : {}".format(node.name))
print("Management IP    : {}".format(management_ip))
print()

gpu1 = node.components[component_name]
print("NVMe Name        : {}".format(gpu1.name))
print("Details          : {}".format(gpu1.details))
print("Units            : {}".format(gpu1.get_property(pname='capacity_allocations').unit))
print("Model            : {}".format(gpu1.model))
print("Type             : {}".format(gpu1.get_property(pname='type')))

## Setup SSH Connection for Commands

Setup <code>paramiko</code> to send commands to the node using <code>ssh</code>.

In [ ]:
import paramiko

#Setup connection to node1
key = paramiko.RSAKey.from_private_key_file(ssh_key_file_priv)
client1 = paramiko.SSHClient()
client1.load_system_host_keys()
client1.set_missing_host_key_policy(paramiko.MissingHostKeyPolicy())

client1.set_missing_host_key_policy(paramiko.AutoAddPolicy())
client1.connect(management_ip,username='ubuntu',pkey = key)

## Configure the GPUs


### NVMe PCI Device

Run the command <code>lspci</code> to see your GPU PCI device(s). This is the raw GPU PCI device that is not yet configured for use.  You can use the GPUs as you would any GPUs.

An example of using the GPUs is coming soon.

View node1's GPU

In [ ]:
stdin, stdout, stderr = client1.exec_command('lspci')

print (str(stdout.read(),'utf-8').replace('\\n','\n'))

## Cleanup Your Experiment

In [ ]:
return_status, result = slice_manager.delete(slice_object=slice)

print("Response Status {}".format(return_status))
print("Response received {}".format(result))